# Contents List

01. Import libraries and data
02. Aggregate mean orders by department_id
03. Analyze results
04. Use transform and loc to create loyalty flag
05. Check stats for each loyalty group
06. Create spending flag
07. Create order frequency flag
08. Export data

# 01. Import libraries and data

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# create shortcut for data imports
path = r'C:\Users\jacym\Desktop\Career Foundry projects\04-2023 Instacart basket analysis'

In [3]:
# import merged data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data','Prepared data', 'orders_products_merged_expanded.pkl'))

In [ ]:
# check imported data
ords_prods_merge.shape

In [4]:
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regular day,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slow day,Average orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slow day,Most orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slow day,Average orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slow day,Most orders


In [ ]:
# import wrangled departments list 
depts_list = pd.read_csv(os.path.join(path, '02 Data', 'Prepared data', 'departments_wrangled.csv'), index_col = 0)

Note to self: I had a lot of problems with the above task. I kept getting a double index column with different numbers in each bc of zero counting, or just one index with frozen labeled as zero instead of 1. I fixed this by re-exporting the depts wrangled file *without* the index=False clause that I had before, then importing it here with index_col=0 (index_col=False did not work!)

In [ ]:
depts_list

In [ ]:
data_dict = depts_list.to_dict('index')

In [ ]:
data_dict

# 02: Aggregate mean orders by department id

In [5]:
# groupby department id, calculate mean order number for each group
ords_prods_merge.groupby('department_id')['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

# 03. Analyze result

Observations: The results are quite different from what I got aggregating the subset of 1,000,000 rows, which suggests that data wasn't representative. That isn't surprising considering it included items in fewer than half the categories.
Going off this aggregation, I can see the most frequently (classifiable) ordered items are in the bulk and babies departments, suggesting that many Instacart orders come from households with babies and people who like to buy in bulk. On the low end are alcohol and pets, which makes sense -- not everyone has a pet, and alcohol is probably bought less frequently than food and household products. Interestingly, frozen food is also on the low end for orders.

# 04. Use transform and loc to create loyalty flag

In [6]:
# transform: create new column max_order, transform it with max order number for each user id
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
# check output
ords_prods_merge.head()

In [7]:
# loc: create loyalty flag column with three labels, each in separate cell
# label 1: loyal customer
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [8]:
# label 2: regular customer
ords_prods_merge.loc[(ords_prods_merge['max_order'] > 10) & (ords_prods_merge['max_order'] <= 40), 'loyalty_flag'] = 'Regular customer'

In [9]:
# label 3: new customer
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [ ]:
# check output
ords_prods_merge.head(50)

# 05. Check stats for each loyalty group

In [11]:
# groupby loyalty flag column and calculate basic price stats for each group: mean, min max
ords_prods_merge.groupby('loyalty_flag').agg({'user_id': ['nunique']})


,user_id
,nunique
loyalty_flag,
Loyal customer,17017
New customer,112328
Regular customer,76864


In [ ]:
# to see price range makeup for each loyalty group, groupby both loyalty flag AND price range loc, then aggregate count for each subgroup
ords_prods_merge.groupby(['loyalty_flag', 'price_range_loc']).agg({'price_range_loc': ['count']})

In [ ]:
# loyal: 10284093 products, 1.15% high, 31.27% low, 67.57% mid
# new: 6243990 products, 1.35% high, 31.29% low, 67.35% mid
# regular: 15876776 products, 1.32% high, 31.25% low, 67.43% mid

Observations: The average item price is similar across groups, with loyal customers spending slightly less per item and new customers spending slightly more. I'm not sure if the difference is significant; would need to run an ANOVA.
The price range results also show similarity across groups, though it appears loyal customers are less likely to splurge on high-range products and most likely to purchase mid-range products, and new customers are the most likely group to buy high-range products.

# 06. Create spending flag

In [ ]:
# transform: create new column avg_price, fill('transform') it with avg price for each user id
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [ ]:
# loc: create spending flag column with two labels, each in separate cell
# label 1: low spender
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'

In [ ]:
# label 2: high spender
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

In [ ]:
# count distinct user ids by spending flag (I think this is right?)
ords_prods_merge.groupby(['spending_flag', 'loyalty_flag']).agg({'user_id': ['nunique']})

In [ ]:
low_spend = 202822/(202822+3387)

In [ ]:
low_spend

Observations: Almost all (98.36%) customers are low spenders. Among the high spenders, most are new customers (79.6%) or regular customers (18.36%).

# 07. Create order frequency flag

In [13]:
# transform: create new column order_frequency, fill('transform') it with median days since last order for each user id
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [ ]:
# check output
ords_prods_merge.head()

In [14]:
# loc: create frequency flag column with three labels, each in separate cell
# label 1: non-frequent customer
ords_prods_merge.loc[ords_prods_merge['order_frequency'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [15]:
# label 2: regular customer
ords_prods_merge.loc[(ords_prods_merge['order_frequency'] > 10) & (ords_prods_merge['order_frequency'] <= 20), 'frequency_flag'] = 'Regular customer'

In [16]:
# label 3: frequent customer
ords_prods_merge.loc[ords_prods_merge['order_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [ ]:
# count distinct user ids by order frequency flag and loyalty flag
ords_prods_merge.groupby(['loyalty_flag', 'frequency_flag']).agg({'user_id': ['nunique']})

In [17]:
# count distinct customers by frequency flag
ords_prods_merge.groupby(['frequency_flag']).agg({'user_id': ['nunique']})

,user_id
,nunique
frequency_flag,
Frequent customer,86596
Non-frequent customer,59619
Regular customer,59993


In [ ]:
# run frequency check for frequency flag -- which group has ordered the most items?
ords_prods_merge['frequency_flag'].value_counts()

Observations: Frequent customers make up the highest share of customers and account for the highest share of items ordered. 
Looking at the groups on a more granular level (divided both by loyalty and frequency), the largest single group of customers is new, non-frequent customers. Loyal and regular customers tend to order frequently.
Note: The customer sum was off by a few thousand depending on how I parsed this. Maybe because of missing data (first orders)?

# 08. Export data

In [ ]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_grouped.pkl'))